In [ ]:
import time
import requests
from pprint import pprint
import os

cfg = {}
cfg['tileServer'] = 'http://192.168.44.128/hot/{z}/{x}/{y}.png'
cfg['z-range'] = [11, 17]
cfg['minLat'] = 31.23
cfg['maxLat'] = 32.62
cfg['minLng'] = 118.37
cfg['maxLng'] = 119.23
cfg['outFolderRoot'] = 'D:/Dev/tmp/Tile/'
cfg['outFolder'] = cfg['outFolderRoot'] + 'files/'

cfg['defaultSeaPng'] = cfg['outFolderRoot'] + 'defaults/sea.png'
cfg['defaultSeaContent'] = None
cfg['defaultLandPng'] = cfg['outFolderRoot'] + 'defaults/land.png'
cfg['defaultLandContent'] = None
cfg['metaDir'] = cfg['outFolderRoot'] + '/meta/'

In [ ]:
def myprint(s):
    enable_log = True
    if enable_log:
        print(s)

def readDefaults():
    pathname = cfg['defaultSeaPng']
    statinfo = os.stat(pathname)
    if statinfo.st_size <= 0:
        raise Exception('Invalid default PNG: ' + pathname)
    with open(pathname, 'rb') as f:
        cfg['defaultSeaContent'] = f.read(statinfo.st_size)

    pathname = cfg['defaultLandPng']
    statinfo = os.stat(pathname)
    if statinfo.st_size <= 0:
        raise Exception('Invalid default PNG: ' + pathname)
    with open(pathname, 'rb') as f:
        cfg['defaultLandContent'] = f.read(statinfo.st_size)


def download(z, x, y):
    url = cfg['tileServer']
    url = url.replace('{z}', str(z)).replace('{x}', str(x)).replace('{y}', str(y))
    #myprint(url)

    res1 = requests.get(url, verify=False)
    if res1.status_code != 200:
        raise Exception('Error: 200 expected!')

    if (res1.content == cfg['defaultSeaContent']):
        return "sea"
    elif (res1.content == cfg['defaultLandContent']):
        return "land"
    else:
        pathname = cfg['outFolder'] + str(z) + '-' + str(x) + '-' + str(y) + '.png'
        with open(pathname, 'wb') as f:
            f.write(res1.content)
        return "png"

readDefaults();
#download(15, 27192, 13301)

In [ ]:
import math

def long2tilex(lon, z):
    return math.floor((lon + 180.0) / 360.0 * math.pow(2.0, z))

def lat2tiley(lat, z):
    return math.floor(
        (1.0 - math.log(math.tan(lat * math.pi/180.0) + 1.0 / math.cos(lat * math.pi/180.0)) / math.pi) / 2.0 * math.pow(2.0, z))


def download_z(z):
    print('')
    print('--------------- z = ' + str(z) + ' ---------------')
    y1 = lat2tiley(cfg['minLat'], z)
    y2 = lat2tiley(cfg['maxLat'], z)
    x1 = long2tilex(cfg['minLat'], z)
    x2 = long2tilex(cfg['maxLat'], z)

    if (y1 > y2): (y1, y2) = (y2, y1)
    if (x1 > x2): (x1, x2) = (x2, x1)

    with open(cfg['metaDir'] + str(z) + '.meta', 'w') as f:
        tag = ''
        for x in range(x1, x2 + 1):
            myprint('downloading: x = ' + str(x))

            for y in range(y1, y2 + 1):
                while True:
                    try:
                        tag = download(z, x, y)
                        break
                    except:
                        time.sleep(0.5)

                if tag != 'png':
                    f.write(str(x) + ',' + str(y) + ',' + tag + '\n')

download(0, 0, 0)

In [ ]:
download_z(11)

In [ ]:
download_z(12)

In [ ]:
download_z(13)

In [ ]:
download_z(14)

In [ ]:
download_z(15)

In [ ]:
download_z(16)

In [ ]:
download_z(17)